In [1]:
# Import necessary libraries for the project.
import pandas as pd  # For data manipulation and analysis.
import os            # For interacting with the operating system.

In [ ]:
# source: https://educacion.gob.ec/base-de-datos/

In [2]:
# Initialize project-specific variables and data structures.
carpeta_inicio            = "Inicio"         # Define the starting folder for your project.
nombre_columna_distintiva = "Cod_Provincia"  # Define a distinctive column name in your data.
encontrado_provincia      = False            # Initialize a flag for province detection.

# Prepare for working with CSV files.
archivos_csv = [archivo for archivo in os.listdir(carpeta_inicio) if archivo.endswith(".csv")]

# Create empty DataFrames for storing data.
provincias_unicas = pd.DataFrame(columns=["Cod_Provincia", "Provincia"])
resultado_global  = pd.DataFrame()

In [3]:
# Iterate over each CSV file in the list.
for archivo in archivos_csv:
    # Extract a distinctive column name from the file name.
    nombre_columna = archivo.split("_")[-1].split(".csv")[0]
    
    # Build the full path to the CSV file.
    ruta_archivo = os.path.join(carpeta_inicio, archivo)

    # Find the row with the header containing the distinctive column name.
    header_row = next(i for i, linea in enumerate(open(ruta_archivo, 'r', encoding='ISO-8859-1')) if nombre_columna_distintiva in linea)

    # Read the CSV file into a DataFrame, skipping the header row.
    df = pd.read_csv(ruta_archivo, skiprows=header_row, encoding='ISO-8859-1', delimiter=';')

    # Rename a specific column if it exists.
    if "Total Estudiantes" in df.columns:
        df.rename(columns={"Total Estudiantes": "Total_Estudiantes"}, inplace=True)

    # Perform a groupby operation to sum the 'Total_Estudiantes' column by 'Cod_Provincia'.
    resultado = df.groupby('Cod_Provincia')['Total_Estudiantes'].sum().astype(int).astype(str).reset_index()
    resultado = resultado.rename(columns={'Total_Estudiantes': f'{nombre_columna}'})
    
    # Check if the 'Provincia' column exists and populate 'provincias_unicas' if not already done.
    if "Provincia" in df.columns and not encontrado_provincia:
        provincias_unicas = df[["Cod_Provincia", "Provincia"]].drop_duplicates()
        encontrado_provincia = True
    
    # Merge the 'resultado' DataFrame with the 'resultado_global' DataFrame.
    if resultado_global.empty:
        resultado_global = resultado
    else:
        resultado_global = resultado_global.merge(resultado, on='Cod_Provincia', how='outer')

In [4]:
# Merge 'resultado_global' with 'provincias_unicas' on 'Cod_Provincia' using a left join.
resultado_provincias = resultado_global.merge(provincias_unicas, on='Cod_Provincia', how='left')

# Apply a lambda function to the 'Provincia' column to extract the first element if it's a list.
resultado_provincias['Provincia'] = resultado_provincias['Provincia'].apply(lambda x: x[0] if isinstance(x, list) else x)

# Filter out rows with 'Cod_Provincia' equal to 90.
resultado_provincias = resultado_provincias[resultado_provincias['Cod_Provincia'] != 90]

# Reorder the columns in alphabetical order.
resultado_provincias = resultado_provincias.reindex(sorted(resultado_provincias.columns), axis=1)

# Rearrange the columns to have 'Cod_Provincia' and 'Provincia' first, followed by the rest.
resultado_provincias = resultado_provincias[['Cod_Provincia', 'Provincia'] + [col for col in resultado_provincias.columns if col not in ['Cod_Provincia', 'Provincia']]

# Optionally, you can print the resulting DataFrame.
# print(resultado_provincias)

    Cod_Provincia                       Provincia 2009-2010Inicio  \
0               1                           AZUAY          200034   
1               2                         BOLIVAR           59292   
2               3                          CAÃAR           71945   
3               4                          CARCHI           45972   
4               5                        COTOPAXI          120260   
5               6                      CHIMBORAZO          137144   
6               7                          EL ORO          172640   
7               8                      ESMERALDAS          183636   
8               9                          GUAYAS          961491   
9              10                        IMBABURA          120771   
10             11                            LOJA          133579   
11             12                        LOS RIOS          234018   
12             13                          MANABI          398805   
13             14                 

In [5]:
# Save the 'resultado_provincias' DataFrame to a CSV file called 'resultadosF.csv' without including the index.
resultado_provincias.to_csv('resultadosF.csv', index=False)